Detect and trace protrusions in T-cells through a z-stack. With this script we can find out, how long protrusions are and how they are distributed. The script consists of 9 steps:

* Step 1: Load the image and do preprocessing
* Step 2: Find the zero layer
* Step 3: Filtering the data
* Step 4: Segment the binary filtered images
* Step 5: Track objects through stack
* Step 6: Calculate the length of each protrusion
* Step 7: Plot tracked protrusions in image
* Step 8: Save the image as a tiff
* Step 9: Create/save the overlay with original image

For each step, a short summary can be found below. If the step requires any parameter, they are defined at the beginning of the step.

The script requires the data to be saved inside of a hdf5 file. Here is how such a hdf5 file can be created using imagej:

* Open the file that needs to be analyzed
* File > Save as > HDF5 (new or replace)
* Choose 'standard' and save file

The 'standard' option is required in order to have the correct keys inside of the file.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import h5py

from skimage import filters
from skimage import measure

from scipy.ndimage import gaussian_filter as gauss

import bokeh
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from ipywidgets import interact
output_notebook()

from matplotlib import cm

import tifffile

<b>This script has been written for the following packages:</b> <br>
<pre>
python:       3.7.5

numpy:        1.16.4 
h5py:         2.10.0 
skimage:      0.16.2 
scipy:        1.4.1  
bokeh:        2.0.0  
ipywidgets:   7.5.1  
matplotlib:   3.1.1  
tifffile:     2020.2.16  
</pre>

# Step 1: Load the image and do preprocessing

In [ ]:
filename = "test"      # Filename without ending. E.g. "test"
z_res    = 0.3         # microns per pixel in z direction
xy_res   = 0.08        # microns per pixel in x and y direction
flag     = True        # If flag is True, than flip data in z direction.

In [ ]:
f        = h5py.File(filename + ".h5",'r')

In [ ]:
data     = np.transpose(f['t0/channel0'],[1,2,0]) # Only channel0 contains the objects in question. This may differ for other types of images

In [ ]:
# Make lowest slide last in stack. 
if flag:
    data = np.flip(data,2)

In [ ]:
# This block is only used for plotting and can savely be ignored, if it does not run without errors
index = 0
p     = figure(title="Channel 1 of data", plot_height=750, plot_width=750)
r     = p.image(image=[data[:,:,index]], x=[0], y=[0], dw=[10], dh=[10], palette="Greys256")

def update(i=0):
    r.data_source.data['image'] = [data[:,:,i]]
    push_notebook()
    
show(p, notebook_handle=True)
interact(update, i=(0,data.shape[2]-1))

# Step 2: Find the zero layer

This is the layer where the surface of the cell hits the nanopore grid. The layer with the highest illuminance is set as the start. In case there is an offset, this can be set below.

In [ ]:
offset  = 0 # In case the data requires an offset between brightes layer and start of surface, this can be set here.

In [ ]:
z_layer = np.argmax(np.mean(data,axis=(0,1))) + offset

In [ ]:
# This block is only used for plotting and can savely be ignored, if it does not run without errors

plt.rcParams.update({'font.size':22})
plt.figure(figsize=(9,5))
plt.plot(np.mean(data,axis=(0,1)))
plt.title('Illuminance per layer')
plt.xlabel('Layer index')
plt.ylabel('Illuminance')
plt.axvline(z_layer)
plt.grid(True)
plt.show()

# Step 3: Filtering the data

The data is filtered by first normalizing the total intesity. After normalization, the stack has 0 mean and standard deviation of 1. 

Filtering is done by looking at the difference of Gaussians (Band-pass filter). It is an approximation of a Maxican Hat function and enables us to detect the protrusions in each layer. It has two parameters, which are the standard deviations of the two Gaussians.

The filtered layers are thresholded. The threshold value can also be set.

In [ ]:
filt_in  = 1.5  # Standard deviation of smaller Gaussian in pixel
filt_out = 2.5  # Standard deviation of smaller Gaussian in pixel
thr      = 1.   # Threshold

# For plotting:
plotting_flag = False # If True, the thresholded images are shown. Otherwise, the background is black.

In [ ]:
norm_data =      data - np.mean(data[:,:,z_layer:])
norm_data = norm_data / np.std(norm_data[:,:,z_layer:])

In [ ]:
filt_data = np.copy(norm_data[:,:,z_layer:])
for i in range(filt_data.shape[2]):
    filt_data[:,:,i] = gauss(filt_data[:,:,i],filt_in)-gauss(filt_data[:,:,i],filt_out)

thr_data  = (filt_data>thr)*1.

In [ ]:
merged    = np.concatenate((thr_data*(plotting_flag*20.)-10,filt_data),axis=1)

In [ ]:
# This block is only used for plotting and can savely be ignored, if it does not run without errors

index     = 0
p         = figure(title="Channel 1 of data", plot_height=500, plot_width=1000)

col_map   = bokeh.models.mappers.LinearColorMapper(palette='Greys256',low=-10,high=10.)

r         = p.image(image=[merged[:,:,index]], x=[0], y=[0], dw=[10], dh=[10], color_mapper=col_map)

def update(i=0):
    r.data_source.data['image'] = [merged[:,:,i]]
    push_notebook()
    
show(p, notebook_handle=True)
interact(update, i=(0,merged.shape[2]-1))

# Step 4: Segment the binary filtered images

In this step, all 'pixel island' in a layer are detected. For each island, only the mean location is getting saved. Islands that are to large or small are thrown out, since they are artifacts.

In [ ]:
island_min_size = 1
island_max_size = 100

In [ ]:
l_labels = []
for i in range(filt_data.shape[2]):
    l_labels.append([])
    labels  = measure.label(thr_data[:,:,i])
    for j in range(np.max(labels)+1):
        vec = np.argwhere(labels==j)
        if vec.shape[0] < island_max_size and vec.shape[0] >= island_min_size: # This makes sure the area is not too large or too small
            l_labels[i].append(np.mean(vec,0))

# Step 5: Track objects through stack

After finding the pixel islands in each frame, they are stichted together (tracked) through all layers. The result represents the protrusions.

For this, we start out at the last frame. Each segment is an object. For each object, find the closest object in the frame above. If the the object is closer than max_dist, they belong together. Objects can be missing in multiple frames; up to 'n_miss' can be missed. If no other object is close by, create a new object (i.e. protrusion).

After the protrusions got stiched together, the list of protrusions is cleaned up. For this, only protrusions with at least 'min_length' layers are considered. Furthermore, the holes in jumped over frames are filled.

In [ ]:
max_dist   = 0.5 # µm. Maximal allowed shift between the same protrusion in two consecutive frames
n_miss     = 2   # Number of stacks that can be jumped over while still considering the same protrusion
min_length = 2   # Minimal length for an object to count as a protrusion

location of j.th potential fiber in frame i: l_labels[i][j] in R^2

In [ ]:
fibers = []
for i in range(len(l_labels)-1,-1,-1):    
    # Create list of all potential fibers
    indices     = []
    locs        = []
    depths      = []
    for j in range(len(fibers)):
        fiber_x = fibers[j][-1][0]
        fiber_y = fibers[j][-1][1]
        fiber_z = fibers[j][-1][2]
        if fiber_z-n_miss <= i:
            indices.append(j)
            locs.append(np.array([fiber_x,fiber_y]))
            depths.append(fiber_z)
    locs        = np.array(locs)
    indices     = np.array(indices)
    depths      = np.array(depths)
    
    # Add the elements to the list, provided, it fits to a fiber. If not, make a new fiber out of it  
    for j in range(len(l_labels[i])):
        flag = False
        if locs.shape[0] > 0:
            loc  = l_labels[i][j]

            dist = (locs - np.expand_dims(loc,0))**2
            dist = np.sum(dist,1)
            dist_index = np.argsort(dist)[0]
            if dist[dist_index]*xy_res <= max_dist:
                
                fibers[indices[dist_index]].append([loc[0],loc[1],i])
                
                flag = True
                locs = np.delete(locs,dist_index,0)
                indices = np.delete(indices,dist_index,0)

        if not flag:
            loc  = l_labels[i][j]
            new_list = []
            new_list.append([loc[0],loc[1],i]) # These are the x,y,z locations of the fiber
            fibers.append(new_list)

In [ ]:
print('Detected protrusions pre clean-up: ' + str(len(fibers)))

In [ ]:
# Clean up the fibers. Two things. First, throw out fibers with length 2 or shorter. Second, interpolate missing layers
clean_fiber = []
for fiber in fibers:
    # Kick out fiber with wrong length
    if len(fiber) <= 2:
        continue
    
    # Interpolation
    temp_fiber = []
    temp_fiber.append(fiber[0])
    for i in range(1,len(fiber)):
        old_height = temp_fiber[-1][2]
        new_height = fiber[i][2]
        if old_height-1 == new_height:
            pass
        else:
            old_pos     = np.array([temp_fiber[-1][0],temp_fiber[-1][1]])
            new_pos     = np.array([fiber[i][0],fiber[i][1]])
            diff_height = old_height - new_height # is bigger than 1
            
            for height in range(1,diff_height):
                pos = old_pos + (new_pos-old_pos)*height/diff_height
                temp_fiber.append([pos[0],pos[1],old_height-height])
        temp_fiber.append(fiber[i])
    clean_fiber.append(temp_fiber)
fibers = clean_fiber

In [ ]:
print('Detected protrusions post clean-up: ' + str(len(fibers)))

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(np.sort(np.array([len(fibers[i]) for i in range(len(fibers))]))[::-1])
plt.title('Sorted Protrusion Length')
plt.grid(True)
plt.xlabel('Protrusion index (sorted)')
plt.ylabel(' Number of frames \n containing protrusion ')
plt.show()

In [ ]:
num_fiber_per_frame = np.zeros(len(l_labels))
for i in range(len(l_labels)):
    for j in range(len(fibers)):
        for k in range(len(fibers[j])):
            if fibers[j][k][2] == i:
                num_fiber_per_frame[i] += 1
                break
            
plt.figure(figsize=(10,5))
plt.plot(num_fiber_per_frame)
plt.title('Number of protrusions in each frame')
plt.xlabel('Protrusion index')
plt.ylabel('Number of protrusions')
plt.grid(True)
plt.show()

# Step 6: Calculate the length of each protrusion

In this step, we create a histogram that shows us how the length of the protrusions is distributed.

In [ ]:
length = [len(fibers[i]) for i in range(len(fibers))]
bins   = (np.arange(np.max(length)+1)+0.5)*z_res
length = np.array(length)*z_res  # Transform length from pixel distance to distance in um
plt.figure(figsize=(20,10))
plt.hist(length, bins=bins)
plt.title('Histogram of protrusion length')
plt.xlabel('Length of protrusion in microns')
plt.ylabel(' Number of protrusions with given length ')
plt.grid(True)
plt.show()

# Step 7: Plot tracked protrusions in image

In this step, the protrusions are ploted for each frame in the stack. The color of the protrusion is either random or depends on the relative length of the protrusion.

In [ ]:
random_color = False # If True, the color of a protrusion is random. Otherwise, it depends on the length.

In [ ]:
def get_color(fibers,i,random=False):
    if random:
        vec = np.abs(np.random.randn(3))
        vec = vec / np.sqrt(np.sum(np.square(vec)))
        return vec*20-10
    else:
        l = [len(fibers[j]) for j in range(len(fibers))] # all the lenghts
        l = np.array(l)
        r = l[i]
        l = np.unique(l)
        r = (1.*np.sum((l<=r)*1.))/l.shape[0] # Relative length
        
        # Choose the color function here. E.g.: autumn,summer,jet (https://matplotlib.org/tutorials/colors/colormaps.html)
        vec = cm.jet(r)
        
        return np.array(vec[:3])*20-10

In [ ]:
col_merged          = np.tile(np.expand_dims(np.clip(merged,-10,10),2),[1,1,4,1])
col_merged[:,:,3,:] = 10.

for i in range(len(fibers)):
    col       = get_color(fibers,i,random=random_color)
    for j in range(len(fibers[i])):
        x_min = max(0,int(fibers[i][j][0])-2)
        y_min = max(0,int(fibers[i][j][1])-2)
        x_max = min(x_min+5,col_merged.shape[0])
        y_max = min(y_min+5,col_merged.shape[0])
        col_merged[x_min:x_max,y_min:y_max,:3,fibers[i][j][2]] = np.expand_dims(np.expand_dims(col,0),0)

In [ ]:
# This block is only used for plotting and can savely be ignored, if it does not run without errors
index     = 0
p         = figure(title="Channel 1 of data", plot_height=500, plot_width=1000)

r         = p.image_rgba(image=[((col_merged[:,:,:,index]/20+0.5)*255).astype(np.uint8)], x=0, y=0, dw=10, dh=10)

def update(i=0):
    r.data_source.data['image'] = [((col_merged[:,:,:,i]/20+0.5)*255).astype(np.uint8)]
    push_notebook()
    
show(p, notebook_handle=True)
interact(update, i=(0,col_merged.shape[3]-1))

# Step 8: Save the image as a tiff

Here, we save the results from step 7 in a tiff file. The file in which it is saved has the identical name as the original file with the ending: "_large.tif".

In [ ]:
# Transform from double to integer (image has now values in [0,255])
trans_im = (((col_merged+10)/20)*(2**8-1)).astype(np.uint8)

In [ ]:
# Change order of axis
trans_im = np.transpose(trans_im,[3,0,1,2])

In [ ]:
tifffile.imsave(filename + "_large.tif",trans_im[:,:,:,:3])

# Step 9: Create/save the overlay with original image

Here, we do the same thing as in step 7 and 8, except that we overlay the protrusions on the original image. The resulting image is saved with the file name of the orginal file with the ending: "_SI.tif".

In [ ]:
random_color = False # If True, the color of a protrusion is random. Otherwise, it depends on the length.

In [ ]:
def get_color_int(fibers,i,random=False):
    vec = (get_color(fibers,i,random)+10)/10
    return (vec*(2**8-1)).astype(np.uint8)

In [ ]:
# Create the overlay of original images and colored points
SI_im          = np.tile(np.expand_dims(data[:,:,z_layer:],2),[1,1,3,1])

# If you want a black image to create alpha channel images in imagej:
#SI_im          = np.tile(np.expand_dims(np.zeros_like(data[:,:,z_layer+int((med_filt-1)/2):]),2),[1,1,3,1])

for i in range(len(fibers)):
    col       = get_color_int(fibers,i,random=random_color)
    for j in range(len(fibers[i])):
        x_min = max(0,int(fibers[i][j][0]))
        y_min = max(0,int(fibers[i][j][1]))
        x_max = min(x_min+3,col_merged.shape[0])
        y_max = min(y_min+3,col_merged.shape[0])
        SI_im[x_min:x_max,y_min:y_max,:,int(fibers[i][j][2])] = np.expand_dims(np.expand_dims(col,0),0)

In [ ]:
# Change order of axis
SI_im = np.transpose(SI_im.astype(np.uint8),[3,0,1,2])

In [ ]:
tifffile.imsave(filename + "_SI.tif",SI_im)